In [1]:
# Import Dependencies
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt 
import gmaps
from configMiriam import gkey
import csv
gmaps.configure(api_key=gkey)


In [2]:
# Create a reference the CSV file desired
csv_path = "Output/2018 DC Parking Data.csv"

# Read the CSV into a Pandas DataFrame
Parking2018 = pd.read_csv(csv_path)

# Print the first five rows of data
Parking2018.head()


,Unnamed: 0,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,0,-77.033642,38.901937,4204073,P039,PARK AT EXPIRED METER,900 BLOCK 15TH ST NW EAST SIDE,TN,800554,471.0,397082,137120,2018-04-02T10:44:00.000Z,2018,4,2,10,44,Monday,0
1,1,-77.075907,38.935698,4204074,P003,RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT,3800 BLOCK NEWARK ST NW SOUTH SI*,MD,803108,2534.0,393419,140870,2018-04-02T12:11:00.000Z,2018,4,2,12,11,Monday,0
2,2,-77.034818,38.900766,4204075,P302,GOVERNMENT VEHICLE PARKING ONLY,800 BLOCK VERMONT AVE NW EAST S*,MD,812519,10761.0,396980,136990,2018-04-02T17:11:00.000Z,2018,4,2,17,11,Monday,0
3,3,-77.036547,38.900783,4204076,P002,STAND OR PARK IN ALLEY,800 BLOCK 16TH ST NW WEST SIDE,MD,813038,11213.0,396830,136992,2018-04-02T00:00:00.000Z,2018,4,2,0,0,Monday,0
4,4,-77.061715,38.943166,4204077,P014,EMERGENCY NO PARKING,2900 BLOCK VAN NESS ST NW NORTH *,DC,812163,10439.0,394650,141698,2018-04-02T10:40:00.000Z,2018,4,2,10,40,Monday,0


In [3]:
# Remove the "Unnamed: 0" column
Parking2018 = Parking2018.drop(columns=["Unnamed: 0"])
Parking2018.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.033642,38.901937,4204073,P039,PARK AT EXPIRED METER,900 BLOCK 15TH ST NW EAST SIDE,TN,800554,471.0,397082,137120,2018-04-02T10:44:00.000Z,2018,4,2,10,44,Monday,0
1,-77.075907,38.935698,4204074,P003,RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT,3800 BLOCK NEWARK ST NW SOUTH SI*,MD,803108,2534.0,393419,140870,2018-04-02T12:11:00.000Z,2018,4,2,12,11,Monday,0
2,-77.034818,38.900766,4204075,P302,GOVERNMENT VEHICLE PARKING ONLY,800 BLOCK VERMONT AVE NW EAST S*,MD,812519,10761.0,396980,136990,2018-04-02T17:11:00.000Z,2018,4,2,17,11,Monday,0
3,-77.036547,38.900783,4204076,P002,STAND OR PARK IN ALLEY,800 BLOCK 16TH ST NW WEST SIDE,MD,813038,11213.0,396830,136992,2018-04-02T00:00:00.000Z,2018,4,2,0,0,Monday,0
4,-77.061715,38.943166,4204077,P014,EMERGENCY NO PARKING,2900 BLOCK VAN NESS ST NW NORTH *,DC,812163,10439.0,394650,141698,2018-04-02T10:40:00.000Z,2018,4,2,10,40,Monday,0


In [4]:
# Find the violations based on state
state_total = Parking2018["RP_PLATE_STATE"].unique()
state_total

array(['TN', 'MD', 'DC', 'VA', 'WI', 'NC', 'TX', 'PA', 'AZ', 'IL', 'FL',
       'DP', 'DE', 'MO', 'NY', 'SC', 'OK', 'CT', 'MA', 'NJ', 'GA', 'IN',
       'MI', 'XX', 'ME', 'US', 'OH', 'AK', 'KS', 'WV', 'CA', 'NV', ' ',
       'AL', 'CO', 'RI', 'WY', 'QC', 'LA', 'OR', 'ON', 'MN', 'MS', 'IA',
       'NM', 'VT', 'WA', 'NH', 'ID', 'KY', 'UT', 'ND', 'HI', 'AR', 'IT',
       'AB', 'NB', 'NE', 'SD', 'MT', 'SK', 'BC', 'YY', 'UK', 'NO', 'FM',
       'CN', 'YT', 'MB', 'MM', 'PE', 'PR', 'NU', 'EC', 'XR', 'NS', 'VI',
       'PQ'], dtype=object)

In [5]:
# Remove the blank state plate values
Parking18 = Parking2018.drop(Parking2018[Parking2018["RP_PLATE_STATE"] == " "].index)

# Round the latitude and longitudes to three decimal places
Parking18 = Parking18.round({"X": 3, "Y":3})

# Display the cleaned data frame 
state_total_clean = Parking18["RP_PLATE_STATE"].value_counts()
state_total_clean.head(15)

MD    461946
DC    385073
VA    308926
PA     16550
FL     14626
NY     12419
NC     11980
NJ     11430
TX      8599
GA      6042
OH      4890
MA      4813
CA      4627
IL      3975
SC      3797
Name: RP_PLATE_STATE, dtype: int64

In [6]:
# Find plates only within the DC, MD & VA area and display the data
Parking18_dmv = Parking18.loc[(Parking18["RP_PLATE_STATE"] == "DC") | (Parking18["RP_PLATE_STATE"] == "MD") |
                           (Parking18["RP_PLATE_STATE"] == "VA"), :]

Parking18_dmv.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
1,-77.076,38.936,4204074,P003,RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT,3800 BLOCK NEWARK ST NW SOUTH SI*,MD,803108,2534.0,393419,140870,2018-04-02T12:11:00.000Z,2018,4,2,12,11,Monday,0
2,-77.035,38.901,4204075,P302,GOVERNMENT VEHICLE PARKING ONLY,800 BLOCK VERMONT AVE NW EAST S*,MD,812519,10761.0,396980,136990,2018-04-02T17:11:00.000Z,2018,4,2,17,11,Monday,0
3,-77.037,38.901,4204076,P002,STAND OR PARK IN ALLEY,800 BLOCK 16TH ST NW WEST SIDE,MD,813038,11213.0,396830,136992,2018-04-02T00:00:00.000Z,2018,4,2,0,0,Monday,0
4,-77.062,38.943,4204077,P014,EMERGENCY NO PARKING,2900 BLOCK VAN NESS ST NW NORTH *,DC,812163,10439.0,394650,141698,2018-04-02T10:40:00.000Z,2018,4,2,10,40,Monday,0
5,-77.043,38.893,4204078,P039,PARK AT EXPIRED METER,1800 BLOCK VIRGINIA AVE NW SOUTH*,MD,800185,159.0,396303,136137,2018-04-02T12:22:00.000Z,2018,4,2,12,22,Monday,0


In [7]:
# Find plates outside of the DC, MD & VA area and display the data
Parking18_no_dmv = Parking18.drop(Parking18[(Parking18["RP_PLATE_STATE"] == "DC") | (Parking18["RP_PLATE_STATE"] == "MD") | 
                                            (Parking18["RP_PLATE_STATE"] == "VA")].index)
Parking18_no_dmv.head()


,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.034,38.902,4204073,P039,PARK AT EXPIRED METER,900 BLOCK 15TH ST NW EAST SIDE,TN,800554,471.0,397082,137120,2018-04-02T10:44:00.000Z,2018,4,2,10,44,Monday,0
8,-77.027,38.892,4204118,P007,PARK IN A BUS ZONE,1200 BLOCK CONSTITUTION AVE NW N*,WI,809136,7822.0,397652,136027,2018-04-02T13:20:00.000Z,2018,4,2,13,20,Monday,0
17,-77.014,38.883,4204127,P037,PARK OVERTIME AT A METER,200 BLOCK E ST SW NORTH SIDE,NC,810968,9402.0,398747,135045,2018-04-04T14:49:00.000Z,2018,4,4,14,49,Wednesday,2
31,-76.981,38.888,4204141,P173,NO PARKING STREET CLEANING,1700 BLOCK INDEPENDENCE AVE SE S*,TX,805605,4805.0,401606,135528,2018-04-04T10:00:00.000Z,2018,4,4,10,0,Wednesday,2
32,-77.105,38.935,4204142,P003,RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT,5100 BLOCK WATSON ST NW NORTH SI*,PA,807121,6102.0,390880,140778,2018-04-04T10:10:00.000Z,2018,4,4,10,10,Wednesday,2


In [8]:
# Number of DC, MD, & VA violations by state plate
dmv_total = Parking18_dmv["RP_PLATE_STATE"].value_counts()
dmv_total

MD    461946
DC    385073
VA    308926
Name: RP_PLATE_STATE, dtype: int64

In [9]:
#DMV Plate, Violation Data
dmv_plates = Parking18_dmv.groupby(["X", "Y"]).count()
dmv_plates_grouped = dmv_plates.reset_index()

# Store latitude and longitude in locations
locations = dmv_plates_grouped[["Y", "X"]]

# Fill NaN values and convert to float and display as dataframe
dmv_count = dmv_plates_grouped["OBJECTID"].astype(float)
dmv_plates_grouped.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.112,38.932,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,-77.112,38.935,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20
2,-77.112,38.936,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130,130
3,-77.111,38.931,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,-77.111,38.932,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [10]:
# DMV Plate Heat Map
fig = gmaps.figure()

# Create heat layer
heat_layer_dmv = gmaps.heatmap_layer(locations, weights=dmv_count, dissipating=False, max_intensity=2500, point_radius=.001)


# Add layer
fig.add_layer(heat_layer_dmv)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Maryland Plate, Violation Data
md_plates = Parking18.loc[Parking18["RP_PLATE_STATE"] == "MD", : ]
md_total = md_plates.groupby(["X", "Y"]).count()
md_total_grouped = md_total.reset_index()

# Store latitude and longitude in locations
locations = md_total_grouped[["Y", "X"]]

# Fill NaN values and convert to float
md_count = md_total_grouped["OBJECTID"].astype(float)
md_total_grouped.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.112,38.935,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,-77.112,38.936,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78
2,-77.111,38.934,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
3,-77.111,38.935,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
4,-77.110,38.933,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [12]:
# Maryland Plate Heat Map
fig = gmaps.figure()

# Create heat layer
heat_layer_md = gmaps.heatmap_layer(locations, weights=md_count, dissipating=False, max_intensity=2500, point_radius=.001)


# Add layer
fig.add_layer(heat_layer_md)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#DC Plate, Violation Data
dc_plates = Parking18.loc[Parking18["RP_PLATE_STATE"] == "DC", : ]
dc_total = dc_plates.groupby(["X", "Y"]).count()
dc_total_grouped = dc_total.reset_index()

# Store latitude and longitude in locations
locations = dc_total_grouped[["Y", "X"]]

# Fill NaN values and convert to float
dc_count = dc_total_grouped["OBJECTID"].astype(float)
dc_total_grouped.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.112,38.932,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,-77.112,38.935,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
2,-77.112,38.936,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
3,-77.111,38.931,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,-77.111,38.932,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [14]:
# DC Plate Violation Heat Map
fig = gmaps.figure()

# Create heat layer
heat_layer_dc = gmaps.heatmap_layer(locations, weights=dc_count, dissipating=False, max_intensity=2500, point_radius=.001)


# Add layer
fig.add_layer(heat_layer_dc)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
#VA Plate, Violation Data
va_plates = Parking18.loc[Parking18["RP_PLATE_STATE"] == "VA", : ]
va_total = va_plates.groupby(["X", "Y"]).count()
va_total_grouped = va_total.reset_index()

# Store latitude and longitude in locations
locations = va_total_grouped[["Y", "X"]]

# Fill NaN values and convert to float
va_count = va_total_grouped["OBJECTID"].astype(float)
va_total_grouped.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,DAY OF WEEK,NUMBERED DAY OF WEEK
0,-77.112,38.935,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
1,-77.112,38.936,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31
2,-77.111,38.935,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
3,-77.110,38.933,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,-77.110,38.935,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
# VA Plate Violation Heat Map
fig = gmaps.figure()

# Create heat layer
heat_layer_va = gmaps.heatmap_layer(locations, weights=va_count, dissipating=False, max_intensity=2500, point_radius=.001)


# Add layer
fig.add_layer(heat_layer_va)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))